In [5]:
from __future__ import print_function
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [6]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

In [10]:
number_maps_1 = 25
number_maps_2 = 50
size_filter_1 = 3
size_filter_2 = 4

In [21]:
# Input layer
x  = tf.placeholder(tf.float32, [None, 784], name='x')
y_ = tf.placeholder(tf.float32, [None, 10],  name='y_')
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Convolutional layer 1
W_conv1 = weight_variable([size_filter_1, size_filter_1, 1, number_maps_1])
b_conv1 = bias_variable([number_maps_1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# Convolutional layer 2
W_conv2 = weight_variable([size_filter_2, size_filter_2, number_maps_1, number_maps_2])
b_conv2 = bias_variable([number_maps_2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print("h_pool2.shape")
print(h_pool2.shape)

# Fully connected layer 1
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*number_maps_2])

print("h_pool2_flat.shape")
print(h_pool2_flat.shape)

# W_fc1 = weight_variable([7 * 7 * number_maps_2, 1024])
# b_fc1 = bias_variable([1024])

W_fc1 = weight_variable([7 * 7 * number_maps_2, 10])
b_fc1 = bias_variable([10])

# h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
y = tf.nn.softmax(tf.matmul(h_pool2_flat, W_fc1) + b_fc1, name = 'y')

# Dropout
keep_prob  = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Fully connected layer 2 (Output layer)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

# y = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2, name='y')

print("h_fc1_drop.shape")
print(h_fc1_drop.shape)
# y = tf.nn.softmax(tf.matmul(h_pool2, W_fc2) + b_fc1, name='y')

# Evaluation functions
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

# Training algorithm
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Training steps
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    max_steps = 1000
    for step in range(max_steps):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        if (step % 100) == 0:
            print(step, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5})
    print(max_steps, sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

h_pool2.shape
(?, 7, 7, 50)
h_pool2_flat.shape
(?, 2450)
h_fc1_drop.shape
(?, 1024)
0 0.0893
100 0.0873
200 0.0859
300 0.0859
400 0.0847
500 0.0827
600 0.0817
700 0.076
800 0.0734
900 0.0721
1000 0.0725
